In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                                   title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/countries-of-the-world-2023                          Global Country Information Dataset 2023             23KB  2023-07-08 20:37:33          12095        495  1.0              
nelgiriyewithana/global-youtube-statistics-2023                       Global YouTube Statistics 2023                      60KB  2023-07-28 15:36:38           4127        144  1.0              
joebeachcapital/global-earth-temperatures                             Global Earth Temperatures                           33KB  2023-08-09 06:21:51            858         31  1.0              
arnavsmayan/netflix-userbase-datase

In [ ]:
!kaggle datasets download -d rishabkoul1/vechicle-dataset

 81% 5.00M/6.21M [00:00<00:00, 35.1MB/s]
100% 6.21M/6.21M [00:00<00:00, 42.1MB/s]


In [ ]:
!unzip "/content/vechicle-dataset.zip" -d "/content/drive/MyDrive/Kaggle Datasets"

In [ ]:
train_dir = "/content/drive/MyDrive/Kaggle Datasets/vechicle_with_train/vechicles/train"
test_dir = "/content/drive/MyDrive/Kaggle Datasets/vechicle_with_train/vechicles/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator = train_datagen.flow_from_directory(
    directory=train_dir, target_size=(150, 150), color_mode='rgb', class_mode='categorical', batch_size=32,
    shuffle=True, seed=2022)

Found 528 images belonging to 10 classes.


In [ ]:
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
validation_generator = test_datagen.flow_from_directory(
    directory=test_dir, target_size=(150, 150), color_mode='rgb', class_mode='categorical',
    batch_size=32, shuffle=True, seed=2022)

Found 74 images belonging to 10 classes.


In [ ]:
import tensorflow as tf

Model Definition

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='selu',
                          kernel_regularizer=tf.keras.regularizers.L1L2()),
    tf.keras.layers.Dense(512, activation='selu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,verbose=2,restore_best_weights=True)
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator,
                    epochs=50,verbose=2, callbacks=[monitor])

Epoch 1/50
17/17 - 235s - loss: 191.5804 - accuracy: 0.1042 - val_loss: 43.0232 - val_accuracy: 0.0405 - 235s/epoch - 14s/step
Epoch 2/50
17/17 - 199s - loss: 17.3732 - accuracy: 0.1383 - val_loss: 6.8622 - val_accuracy: 0.0946 - 199s/epoch - 12s/step
Epoch 3/50
17/17 - 154s - loss: 3.4822 - accuracy: 0.2595 - val_loss: 2.7839 - val_accuracy: 0.2297 - 154s/epoch - 9s/step
Epoch 4/50
17/17 - 150s - loss: 1.2620 - accuracy: 0.5777 - val_loss: 1.7275 - val_accuracy: 0.4054 - 150s/epoch - 9s/step
Epoch 5/50
17/17 - 148s - loss: 0.6010 - accuracy: 0.8466 - val_loss: 2.1928 - val_accuracy: 0.3784 - 148s/epoch - 9s/step
Epoch 6/50
17/17 - 140s - loss: 0.2270 - accuracy: 0.9678 - val_loss: 1.9724 - val_accuracy: 0.5000 - 140s/epoch - 8s/step
Epoch 7/50
17/17 - 137s - loss: 0.1224 - accuracy: 0.9886 - val_loss: 1.8817 - val_accuracy: 0.5000 - 137s/epoch - 8s/step
Epoch 8/50
17/17 - 138s - loss: 0.0445 - accuracy: 0.9981 - val_loss: 1.8170 - val_accuracy: 0.4865 - 138s/epoch - 8s/step
Epoch 9/50

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()